<a href="https://colab.research.google.com/github/jiya2107/Machine-Learning---Practical/blob/main/ML_Experiment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load and preprocess data (same as experiment 3, but keep PROT as target)
try:
    df = pd.read_csv('HepatitisCdata.csv', index_col=0)
except FileNotFoundError:
    print("Please ensure 'HepatitisCdata.csv' is uploaded.")
    exit()

df = df.replace('?', np.nan)
for col in df.columns:
    if col != 'Category' and col != 'Sex':
        df[col] = pd.to_numeric(df[col])
df = df.dropna()
df['Sex'] = df['Sex'].map({'m': 1, 'f': 0})
df = df.drop('Category', axis=1) # Drop classification target

# Define X and y for regression (Predicting PROT)
X = df.drop('PROT', axis=1)
y = df['PROT']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"\n--- {model_name} Results ---")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"R-squared (R2): {r2:.2f}")
    try:
        # For regularized models, show some coefficients
        print("Coefficients (Sample):")
        for feature, coef in zip(X.columns, model.coef_):
            if abs(coef) > 0.01: # Print significant ones
                print(f"  {feature}: {coef:.3f}")
    except AttributeError:
        pass # LinearRegression.coef_ is standard

# --- 1. Linear Regression (No Regularization) ---
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)
evaluate_model(lin_reg, X_test_scaled, y_test, "Linear Regression")

# --- 2. Ridge Regression (L2 Regularization) ---
# alpha is the regularization strength (lambda)
ridge_reg = Ridge(alpha=10.0) # Using a moderate alpha
ridge_reg.fit(X_train_scaled, y_train)
evaluate_model(ridge_reg, X_test_scaled, y_test, f"Ridge Regression (L2, alpha=10.0)")

# --- 3. Lasso Regression (L1 Regularization) ---
# alpha is the regularization strength (lambda)
lasso_reg = Lasso(alpha=0.1) # Using an alpha that might enforce sparsity
lasso_reg.fit(X_train_scaled, y_train)
evaluate_model(lasso_reg, X_test_scaled, y_test, f"Lasso Regression (L1, alpha=0.1)")

print("\nNote: Lasso (L1) can drive some coefficients exactly to zero, performing feature selection.")